In [236]:
import pandas as pd
import seaborn as sns
import datetime
from datetime import *
import sys
import gzip
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pylab
import scipy.stats as stats

In [237]:
plt.style.use('seaborn')
pd.options.mode.chained_assignment = None  # default='warn'

# beer

In [238]:
#load the data beers.csv
df_beers = pd.read_csv('./data/beers.csv')

In [239]:
df_beers.head()

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.0,NaN,NaN,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.5,NaN,NaN,0,NaN


In [240]:
list_considered_variable=['beer_id','beer_name','brewery_id','brewery_name','style',
                          'nbr_reviews','nbr_ratings','avg', 'abv']
df_beers=df_beers[list_considered_variable]

In [241]:
beer_catalog = {
    'Bocks' : ['Bock', 'Doppelbock', 'Eisbock', 'Maibock', 'Weizenbock', 'Maibock / Helles Bock'],

    'Dark Ales' : ['Dubbel', 'Roggenbier', 'Scottish Ale', 'Winter Warmer'],

    'Hybrid Beers' : ['Bière de Champagne / Bière Brut', 'Braggot', 'California Common / Steam Beer', 'Cream Ale'],

    'Pale Ales' : ['American Amber / Red Ale', 'American Blonde Ale', 'American Pale Ale', 'Belgian Blonde Ale',
    'Belgian Pale Ale', 'Bière de Garde', 'English Bitter', 'English Pale Ale', 'English Pale Mild Ale', 'Extra Special / Strong Bitter (ESB)',
    'Grisette', 'Irish Red Ale', 'Kölsch', 'Saison', 'Saison / Farmhouse Ale', 'Belgian Strong Pale Ale', 'American Pale Ale (APA)'],

    'Porters' : ['American Porter', 'Baltic Porter', 'English Porter', 'Imperial Porter', 'Robust Porter ', 'Smoked Porter'],

    'Stouts' : ['American Imperial Stout', 'American Stout', 'English Stout', 'Foreign / Export Stout', 'Irish Dry Stout', 'Oatmeal Stout',
    'Russian Imperial Stout', 'Milk / Sweet Stout', 'American Double / Imperial Stout'],

    'Wheat Beers' : ['American Dark Wheat Beer', 'American Pale Wheat Beer', 'Dunkelweizen', 'Grodziskie', 'Hefeweizen', 'Kristallweizen', 'Witbier', 
    'American Pale Wheat Ale', 'American Dark Wheat Ale', 'Berliner Weissbier', 'Kristalweizen'],

    'Brown Ales' : ['Altbier', 'American Brown Ale', 'Belgian Dark Ale', 'English Brown Ale', 'English Dark Mild Ale', 'Belgian Strong Dark Ale'],

    'Dark Lagers' : ['American Amber / Red Lager', 'Czech Amber Lager', 'Czech Dark Lager', 'European Dark Lager', 'Märzen',
    'Munich Dunkel', 'Rauchbier', 'Schwarzbier', 'Vienna Lager', 'Munich Dunkel Lager', 'Euro Dark Lager'],

    'India Pale Ales' : ['American IPA', 'American Double / Imperial IPA', 'Belgian IPA', 'Black IPA', 'Brut IPA', 'English IPA', 'English India Pale Ale (IPA)', 
    'Imperial IPA', 'Milkshake IPA', 'New England IPA', 'American Black Ale'],

    'Pale Lagers' : ['American Adjunct Lager', 'American Lager', 'Bohemian / Czech Pilsner', 'Czech Pale Lager', 'European / Dortmunder Export Lager',
    'European Pale Lager', 'European Strong Lager', 'Festbier / Wiesnbier', 'German Pilsner', 'Helles', 'Imperial Pilsner', 'India Pale Lager (IPL)',
    'Kellerbier / Zwickelbier', 'Light Lager', 'Malt Liquor', 'Czech Pilsener', 'Euro Pale Lager', 'American Pale Lager', 'German Pilsener', 'Dortmunder / Export Lager', 
    'Euro Strong Lager', 'Munich Helles Lager', 'American Double / Imperial Pilsner', 'American Malt Liquor', 'Märzen / Oktoberfest'],

    'Specialty Beer' : ['Chile Beer', 'Fruit and Field Beer', 'Gruit / Ancient Herbed Ale', 'Happoshu', 'Herb and Spice Beer', 'Japanese Rice Lager',
    'Kvass', 'Low-Alcohol Beer', 'Pumpkin Beer', 'Rye Beer', 'Sahti', 'Smoked Beer', 'Herbed / Spiced Beer', 'Pumpkin Ale', 'Scottish Gruit / Ancient Herbed Ale', 'Low Alcohol Beer'],

    'Strong Ales' : ['American Barleywine', 'American Strong Ale', 'Belgian Dark Strong Ale', 'Belgian Pale Strong Ale', 'English Barleywine', 'English Strong Ale',
    'Imperial Red Ale', 'Old Ale', 'Quadrupel (Quad)', 'Scotch Ale / Wee Heavy', 'Tripel', 'Wheatwine', 'Fruit / Vegetable Beer'],
    
    'Wild/Sour Beers' : ['Berliner Weisse', 'Brett Beer', 'Faro', 'Flanders Oud Bruin', 'Flanders Red Ale', 'Fruit Lambic',
    'Fruited Kettle Sour', 'Gose', 'Gueuze', 'Lambic', 'Wild Ale', 'Lambic - Unblended', 'Lambic - Fruit', 'American Wild Ale'],

    'Weird Cocktail' : ['Black & Tan']
}

In [242]:
beer_catalog_reverse = {}
for key, values in beer_catalog.items():
    for value in values:
            beer_catalog_reverse.setdefault(value, []).append(key)

In [243]:
def category_check(beer_style, beer_catalog):
    '''
    Return the new columns of 15 categories made in the website https://www.beeradvocate.com/beer/styles/
    '''
    for category_name, beer_styles in beer_catalog.items():
        if beer_style in beer_styles:
            return category_name

In [244]:
df_beers['category'] = df_beers['style'].apply(lambda beer_style: category_check(beer_style, beer_catalog))

In [245]:
df_beers

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_reviews,nbr_ratings,avg,abv,category
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.7,Pale Lagers
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,3.8,Pale Lagers
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.8,Pale Lagers
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.0,Pale Lagers
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.5,Pale Lagers
...,...,...,...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,3,2.71,5.0,Pale Ales
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,1,2,3.10,5.3,Brown Ales
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,0,NaN,NaN,Pale Lagers
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,0,NaN,7.0,Strong Ales


# user

In [246]:
#load the data
df_users = pd.read_csv('./data/users.csv')
print(df_users.describe())
df_users.head(5)

         nbr_ratings    nbr_reviews        joined
count  153704.000000  153704.000000  1.510520e+05
mean       54.605163      16.847876  1.357238e+09
std       252.388790     139.846706  9.195131e+07
min         1.000000       0.000000  8.407944e+08
25%         1.000000       0.000000  1.303121e+09
50%         3.000000       0.000000  1.391944e+09
75%        16.000000       2.000000  1.417691e+09
max     12046.000000    8970.000000  1.501495e+09


,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


In [247]:
#keep user from the USA only
df_us_users = df_users[df_users.location.str.contains('United States', na=False)]

#choose a minimum number of ratings per user, since we want to analyse an evolution
cutoff_low = 500 #subjective, good so ?
print("Cutoff low : ", cutoff_low)

#choose a maximum number of ratings per user, we want to filter our super-users or organisations
cutoff_high = df_us_users.nbr_ratings.quantile(0.99)
print("Cutoff high : ", cutoff_high)

#create a valid user dataframe and a set with the valid user_names
df_valid_us_users = df_us_users[(df_us_users.nbr_ratings < cutoff_high) &
                                (df_us_users.nbr_ratings >= cutoff_low)]
valid_us_users_names = set(df_valid_us_users.user_name)
print("Number of users left : ", len(valid_us_users_names))
print("Total of ratings left : ", df_valid_us_users.nbr_ratings.sum())

Cutoff low :  500
Cutoff high :  1135.0
Number of users left :  2232
Total of ratings left :  1652164


In [248]:
df_valid_us_users

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
8,1131,188,okcnittany.144868,OKCNittany,1.182766e+09,"United States, Oklahoma"
15,732,14,byofb.537281,BYOFB,1.292065e+09,"United States, Wisconsin"
17,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois"
33,1078,10,colsanders123.545441,ColSanders123,1.293620e+09,"United States, Georgia"
35,914,890,jays2629.517433,JayS2629,1.287828e+09,"United States, Alabama"
...,...,...,...,...,...,...
43556,548,21,crafteyedoc.675112,CraftEyeDoc,1.336039e+09,"United States, North Carolina"
43626,630,23,pmoney.592770,pmoney,1.302862e+09,"United States, Illinois"
44819,637,48,aleckpa.541457,aleckpa,1.292843e+09,"United States, North Carolina"
52196,1117,4,dinoblueshoes.607196,DinoBlueShoes,1.309342e+09,"United States, Maine"


# ratings


In [249]:
def parse(filename):
    """
    Parse a txt.gz file and return a generator for it
    Copyright © 2017 Gael Lederrey <gael.lederrey@epfl.ch>
    :param filename: name of the file
    :return: Generator to go through the file
    """
    file = gzip.open(filename, 'rb')
    entry = {}
    # Go through all the lines
    for line in file:
        # Transform the string-bytes into a string
        line = line.decode("utf-8").strip()

        # We check for a colon in each line
        colon_pos = line.find(":")
        if colon_pos == -1:
            # if no, we yield the entry
            yield entry
            entry = {}
            continue
        # otherwise, we add the key-value pair to the entry
        key = line[:colon_pos]
        value = line[colon_pos + 2:]
        entry[key] = value

In [250]:
gen = parse('./data/ratings.txt.gz')

In [251]:
create = False

if create:
    undesired_features = set(['text','review', 'brewery_name', 'brewery_id', 'user_id', 'beer_name'])
    full_ratings_ls = []
    int_cols = ['beer_id']
    float_cols = ['abv','appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']

    review = next(gen)
    cols = [key for key, _ in review.items() if key not in undesired_features]

    for idx, review in enumerate(gen):
        user_name = review['user_name']
        valid_rows = [value for key, value in review.items() if key not in undesired_features]
        if user_name in valid_us_users_names:
            full_ratings_ls.append(valid_rows)

    df_ratings = pd.DataFrame(full_ratings_ls, columns = cols)

    df_ratings[int_cols] = df_ratings[int_cols].astype('int')
    df_ratings[float_cols] = df_ratings[float_cols].astype('float') 
    df_ratings["date"] = pd.to_datetime(df_ratings.date.apply(
        lambda date : datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d')
    ))

    #There is one review in 2000, wich is an outlier
    start_date = pd.to_datetime("2001-01-01")
    df_ratings = df_ratings.loc[df_ratings['date'] >= start_date]   
    df_ratings.to_pickle('./data/ratings_valid_user.pkl')

else:
    df_ratings = pd.read_pickle('./data/ratings_valid_user.pkl')

In [252]:
#add the category of the beer
beers_id_cat = df_beers[['beer_id', 'category']]
df_ratings_cat = pd.merge(df_ratings, beers_id_cat, on = 'beer_id')

In [253]:
df_ratings_cat

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category
0,19827,English Pale Ale,4.8,2014-02-10,OKCNittany,NaN,NaN,NaN,NaN,NaN,3.75,Pale Ales
1,19827,English Pale Ale,4.8,2012-12-02,BYOFB,NaN,NaN,NaN,NaN,NaN,3.50,Pale Ales
2,19827,English Pale Ale,4.8,2012-11-17,ferrari29,3.0,3.5,2.5,2.5,2.5,2.77,Pale Ales
3,19827,English Pale Ale,4.8,2011-07-16,ColSanders123,4.0,3.5,3.0,3.5,3.5,3.48,Pale Ales
4,19827,English Pale Ale,4.8,2011-07-11,JayS2629,3.0,3.5,2.5,3.0,3.0,3.07,Pale Ales
...,...,...,...,...,...,...,...,...,...,...,...,...
1652159,19149,Irish Dry Stout,6.5,2006-02-19,Dithyramb,4.0,3.0,2.0,3.0,3.0,2.96,Stouts
1652160,19142,English India Pale Ale (IPA),5.6,2006-02-19,Dithyramb,3.0,3.0,3.0,2.0,3.0,2.60,India Pale Ales
1652161,19141,American Pale Ale (APA),6.8,2006-02-19,Dithyramb,4.0,4.0,4.0,4.0,4.0,4.00,Pale Ales
1652162,19139,Kölsch,5.0,2006-02-19,Dithyramb,3.0,3.0,3.0,2.5,3.0,2.80,Pale Ales


In [254]:
df_ratings_cat.sort_values(by = "date", inplace=True)
df_ratings_cat["rating_number"] = df_ratings_cat.groupby("user_name").transform("cumcount")

In [255]:
df_filt = df_ratings_cat[df_ratings_cat["rating_number"] < 500]

In [256]:
df_filt

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number
1640018,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0
311428,3558,Czech Pilsener,5.00,2001-06-18,Mark,4.00,4.00,3.00,4.00,4.00,3.90,Pale Lagers,0
17313,73,Milk / Sweet Stout,5.20,2001-06-18,Mark,5.00,5.00,5.00,5.00,5.00,5.00,Stouts,1
136697,261,Vienna Lager,4.48,2001-06-24,Mark,4.00,3.00,3.00,3.00,3.00,3.06,Dark Lagers,2
442671,87,American Pale Ale (APA),5.40,2001-06-24,Mark,4.00,4.00,1.00,3.00,3.00,3.10,Pale Ales,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350352,205333,Belgian Pale Ale,5.50,2017-07-26,Blackop555,4.00,3.25,4.00,3.25,3.50,3.42,Pale Ales,495
976033,133247,American IPA,6.50,2017-07-26,Blackop555,3.75,4.25,3.75,4.00,4.00,4.02,India Pale Ales,496
1354178,11947,Fruit / Vegetable Beer,5.50,2017-07-27,Blackop555,3.75,3.50,3.25,3.25,3.25,3.34,Strong Ales,497
1358394,272569,Witbier,4.80,2017-07-27,Blackop555,3.75,4.00,3.75,3.50,3.75,3.71,Wheat Beers,498


In [257]:
#create a column number of days active
number_of_day_active=df_filt.groupby(['user_name']).date.unique()
number_of_day_active.name='Number'
number_of_day_active=pd.DataFrame(number_of_day_active)
number_of_day_active['Number of day active']=number_of_day_active.apply(lambda g : len(g.Number),axis=1)
del number_of_day_active['Number']
df_number_active = df_filt.merge(number_of_day_active,right_on='user_name', left_on = 'user_name')
df_number_active = df_number_active[df_number_active["Number of day active"] >= 100]

In [258]:
df_number_active

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active
0,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0,178
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.50,4.00,3.00,4.00,4.00,3.93,Pale Ales,1,178
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.50,4.50,4.00,3.50,4.00,3.95,Stouts,2,178
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.50,3.00,3.00,2.50,2.50,2.73,Pale Lagers,3,178
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.50,4.50,4.50,4.50,3.50,4.30,Brown Ales,4,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115495,205333,Belgian Pale Ale,5.5,2017-07-26,Blackop555,4.00,3.25,4.00,3.25,3.50,3.42,Pale Ales,495,170
1115496,133247,American IPA,6.5,2017-07-26,Blackop555,3.75,4.25,3.75,4.00,4.00,4.02,India Pale Ales,496,170
1115497,11947,Fruit / Vegetable Beer,5.5,2017-07-27,Blackop555,3.75,3.50,3.25,3.25,3.25,3.34,Strong Ales,497,170
1115498,272569,Witbier,4.8,2017-07-27,Blackop555,3.75,4.00,3.75,3.50,3.75,3.71,Wheat Beers,498,170


In [259]:
#Difference in days between fisrt review and last review
diff_rating_per_user=df_number_active.groupby(['user_name']).apply(lambda group : (pd.Series(group.date.max()-group.date.min()).dt.days))

diff_rating_per_user.name='Different days'
diff_rating_per_user=pd.DataFrame(diff_rating_per_user)

df_diff_days = df_number_active.merge(diff_rating_per_user,right_on='user_name', left_on = 'user_name')

#df_diff_days = df_diff_days[df_diff_days["Different days"] > pd.Timestamp(365)]
#df_diff_days.rename = ["0"]
df_diff_days = df_diff_days.rename(columns = {0 : "Different days"})
#only keep user within 1 to 7 year
df_diff_days = df_diff_days[(df_diff_days["Different days"] > 365) & (df_diff_days["Different days"] < 365*7)]
df_diff_days


,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active,Different days
0,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0,178,1310
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.50,4.00,3.00,4.00,4.00,3.93,Pale Ales,1,178,1310
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.50,4.50,4.00,3.50,4.00,3.95,Stouts,2,178,1310
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.50,3.00,3.00,2.50,2.50,2.73,Pale Lagers,3,178,1310
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.50,4.50,4.50,4.50,3.50,4.30,Brown Ales,4,178,1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890495,289531,Altbier,6.5,2017-07-01,Jonathan-Morgan,3.50,3.50,3.50,3.50,3.50,3.50,Brown Ales,495,107,376
890496,289527,American Brown Ale,7.3,2017-07-01,Jonathan-Morgan,3.50,3.75,3.50,3.75,3.50,3.66,Brown Ales,496,107,376
890497,289533,Extra Special / Strong Bitter (ESB),5.0,2017-07-01,Jonathan-Morgan,3.50,3.50,3.50,3.50,3.50,3.50,Pale Ales,497,107,376
890498,289562,American Pale Ale (APA),6.5,2017-07-01,Jonathan-Morgan,3.50,3.75,3.50,3.75,3.50,3.66,Pale Ales,498,107,376


In [260]:
x = df_diff_days.groupby(["user_name", pd.Grouper(freq = "D", key="date")])["beer_id"].count()
x.describe(percentiles=[0.99])

count    363882.000000
mean          2.175156
std           3.968860
min           1.000000
50%           1.000000
99%          13.000000
max         314.000000
Name: beer_id, dtype: float64

In [261]:
x = df_diff_days.groupby(["user_name", pd.Grouper(freq = "M", key="date")], as_index=False)["beer_id"].count()
y = x.groupby("user_name")["beer_id"].max()
#x[x["user_name"] == "-Rick"]
y.describe()

count    1583.000000
mean       70.673405
std        45.732403
min        14.000000
25%        41.000000
50%        56.000000
75%        84.000000
max       389.000000
Name: beer_id, dtype: float64

In [262]:
valid = y[(y <= 70)]
df_max_per_mounth = df_diff_days[df_diff_days["user_name"].isin(set(valid.index))]

In [263]:
df_reviews_final = df_max_per_mounth.copy()

In [264]:
df_reviews_final

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active,Different days
0,167,Witbier,NaN,2001-03-16,frank4sail,4.5,4.5,4.0,5.00,5.0,4.75,Wheat Beers,0,178,1310
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.5,4.0,3.0,4.00,4.0,3.93,Pale Ales,1,178,1310
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.5,4.5,4.0,3.50,4.0,3.95,Stouts,2,178,1310
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.5,3.0,3.0,2.50,2.5,2.73,Pale Lagers,3,178,1310
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.5,4.5,4.5,4.50,3.5,4.30,Brown Ales,4,178,1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888995,260713,American Double / Imperial Stout,13.0,2017-05-23,JLK7299,5.0,4.5,4.5,4.50,4.5,4.53,Stouts,495,206,401
888996,262165,American Strong Ale,12.5,2017-05-28,JLK7299,5.0,5.0,4.5,4.50,4.5,4.65,Strong Ales,496,206,401
888997,282574,American IPA,6.8,2017-05-28,JLK7299,4.0,4.0,4.0,4.00,4.0,4.00,India Pale Ales,497,206,401
888998,283390,American IPA,7.2,2017-05-28,JLK7299,5.0,5.0,4.5,4.25,4.5,4.55,India Pale Ales,498,206,401


In [265]:
def compute_diff_date(user):
    diff  = pd.DataFrame()
    for day in [25, 50, 75, 100, 125, 150, 200, 300, 400, 500]:
        diff[str(day)] = pd.Series(user[user["rating_number"] == day-1]["date"].values -\
                         user[user["rating_number"] == 0]["date"].values).dt.days
    return diff

In [266]:
def compute_diff_date_last(user):
    diff  = pd.DataFrame()
    for day in [400, 450]:
        diff[str(day)] = pd.Series(user[user["rating_number"] == 500 -1]["date"].values -\
                         user[user["rating_number"] == day]["date"].values).dt.days
    return diff

In [267]:
df_reviews_final["date"] = pd.to_datetime(df_reviews_final["date"])

In [268]:
grouped = df_reviews_final.groupby("user_name").apply(compute_diff_date).reset_index()
del grouped["level_1"]
grouped

,user_name,25,50,75,100,125,150,200,300,400,500
0,-Rick,0,17,41,58,66,78,113,158,296,460
1,1000lbgrizzly,32,79,140,224,297,395,597,823,913,1021
2,1424IpA,23,94,117,121,142,161,207,294,394,500
3,2Xmd,49,73,97,141,175,295,387,519,659,806
4,2ellas,11,49,66,68,80,115,291,490,615,690
...,...,...,...,...,...,...,...,...,...,...,...
1046,zenithberwyn,0,0,50,80,125,142,177,361,512,688
1047,zhanson88,32,56,82,114,128,164,216,398,726,1418
1048,zookerman182,44,57,120,148,198,233,286,382,458,617
1049,zoso1967,330,358,384,414,448,482,547,627,768,1000


In [269]:
#grouped = grouped[grouped["150"] < 2*365]

In [270]:
grouped_melt = grouped.copy()
grouped_melt = pd.melt(grouped_melt, id_vars=["user_name"], value_vars=["25", "50", "75", "100", "125", "150", "200", "300", "400", "500"])
grouped_melt.columns = ["user_name", "cat_nb_days","day_diff"]
grouped_melt

,user_name,cat_nb_days,day_diff
0,-Rick,25,0
1,1000lbgrizzly,25,32
2,1424IpA,25,23
3,2Xmd,25,49
4,2ellas,25,11
...,...,...,...
10505,zenithberwyn,500,688
10506,zhanson88,500,1418
10507,zookerman182,500,617
10508,zoso1967,500,1000


In [271]:
grouped.sort_values(by="50", inplace = True)

In [272]:
px.line(grouped, x=range(grouped.shape[0]), y="50")

In [273]:
grouped.sort_values(by="50", inplace = True)
grouped = grouped[grouped["50"] < 500]

In [274]:
grouped_last = df_reviews_final.groupby("user_name").apply(compute_diff_date_last).reset_index()
del grouped_last["level_1"]
grouped_last.sort_values(by="450", inplace = True)
grouped_last

,user_name,400,450
113,ChrisKayler,205,0
166,EnojysAnyBeer,1327,2
657,cosmonick,59,8
732,gibgink,53,11
217,Gushue3,333,12
...,...,...,...
461,Skidz,1273,1051
986,tastybeer,1393,1063
245,Imstillthegman,1243,1082
623,brownbeer,1379,1302


In [275]:
px.line(grouped_last, x=range(grouped_last.shape[0]), y="450")

In [276]:
#filter according to first 50
grouped_last = grouped_last[grouped_last["user_name"].isin(grouped["user_name"])]

In [277]:
grouped_last = grouped_last[grouped_last["450"] < 500]

In [278]:
grouped = grouped[grouped["user_name"].isin(grouped_last["user_name"])]

In [279]:
px.line(grouped_last, x=range(grouped_last.shape[0]), y="450")

In [280]:
px.line(grouped, x=range(grouped.shape[0]), y="50")

In [281]:
fig = px.box(grouped_melt, x="cat_nb_days", y="day_diff")
fig.show()
fig.write_html("day_diff.html")

In [282]:
df_reviews_final = df_reviews_final[df_reviews_final["user_name"].isin(grouped["user_name"])]
#NOW ITS THE REAL FINAL

In [283]:
#df_reviews_final.to_pickle("df_reviews_real_final.pkl")

In [284]:
df_reviews_final

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active,Different days
0,167,Witbier,NaN,2001-03-16,frank4sail,4.5,4.5,4.0,5.00,5.0,4.75,Wheat Beers,0,178,1310
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.5,4.0,3.0,4.00,4.0,3.93,Pale Ales,1,178,1310
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.5,4.5,4.0,3.50,4.0,3.95,Stouts,2,178,1310
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.5,3.0,3.0,2.50,2.5,2.73,Pale Lagers,3,178,1310
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.5,4.5,4.5,4.50,3.5,4.30,Brown Ales,4,178,1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888995,260713,American Double / Imperial Stout,13.0,2017-05-23,JLK7299,5.0,4.5,4.5,4.50,4.5,4.53,Stouts,495,206,401
888996,262165,American Strong Ale,12.5,2017-05-28,JLK7299,5.0,5.0,4.5,4.50,4.5,4.65,Strong Ales,496,206,401
888997,282574,American IPA,6.8,2017-05-28,JLK7299,4.0,4.0,4.0,4.00,4.0,4.00,India Pale Ales,497,206,401
888998,283390,American IPA,7.2,2017-05-28,JLK7299,5.0,5.0,4.5,4.25,4.5,4.55,India Pale Ales,498,206,401


### cluster 50 first

In [285]:
df_to_cluster = df_reviews_final[df_reviews_final["rating_number"] < 50]

In [286]:
user_to_cat = df_to_cluster.groupby(["user_name", "category"])["beer_id"].count()
user_to_cat = user_to_cat.unstack(level=-1)
user_to_cat.fillna(0, inplace = True)

In [287]:
user_to_cat

category,Bocks,Brown Ales,Dark Ales,Dark Lagers,Hybrid Beers,India Pale Ales,Pale Ales,Pale Lagers,Porters,Specialty Beer,Stouts,Strong Ales,Weird Cocktail,Wheat Beers,Wild/Sour Beers
user_name,,,,,,,,,,,,,,,
-Rick,0.0,2.0,0.0,2.0,0.0,29.0,7.0,2.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0
1000lbgrizzly,0.0,5.0,0.0,1.0,0.0,4.0,9.0,7.0,2.0,8.0,6.0,4.0,0.0,3.0,1.0
1424IpA,0.0,0.0,1.0,0.0,0.0,21.0,3.0,1.0,1.0,1.0,12.0,4.0,0.0,1.0,5.0
2Xmd,0.0,2.0,1.0,1.0,0.0,25.0,5.0,1.0,1.0,2.0,9.0,1.0,0.0,2.0,0.0
2ellas,0.0,2.0,1.0,0.0,0.0,31.0,9.0,0.0,0.0,0.0,2.0,4.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zenithberwyn,1.0,1.0,0.0,3.0,0.0,3.0,7.0,10.0,3.0,0.0,14.0,5.0,0.0,3.0,0.0
zhanson88,1.0,5.0,3.0,1.0,0.0,6.0,6.0,2.0,3.0,1.0,12.0,7.0,0.0,3.0,0.0
zookerman182,1.0,3.0,0.0,0.0,0.0,22.0,6.0,0.0,2.0,0.0,9.0,5.0,0.0,1.0,1.0


In [288]:
from sklearn.cluster import KMeans

In [289]:
our_clustering = KMeans(4).fit(user_to_cat.values)

In [290]:
import collections
# for cluster_num in range(4)    
#     np.count_nonzero(our_clustering.labels_ == 0)
count = collections.Counter(our_clustering.labels_)
count

Counter({2: 239, 0: 394, 1: 196, 3: 141})

In [291]:
df_cluster_beer = pd.DataFrame(our_clustering.cluster_centers_.T)
#df_cluster_beer.rename(columns = user_to_cat.columns)
df_cluster_beer.index = user_to_cat.columns
df_cluster_beer["style"] = df_cluster_beer.index
df_cluster_beer = pd.melt(df_cluster_beer, id_vars=["style"], value_vars=[0,1,2,3])
df_cluster_beer.columns = ["style", "cluster", "Number of beer"]
df_cluster_beer

,style,cluster,Number of beer
0,Bocks,0,1.659898
1,Brown Ales,0,3.281726
2,Dark Ales,0,1.654822
3,Dark Lagers,0,1.530457
4,Hybrid Beers,0,0.337563
5,India Pale Ales,0,8.192893
6,Pale Ales,0,9.408629
7,Pale Lagers,0,3.868020
8,Porters,0,2.296954
9,Specialty Beer,0,1.743655


In [292]:
fig = px.line(df_cluster_beer, x="style", y="Number of beer", color="cluster", title="Centroids obtained with Kmeans (k=4) of the first 50 ratings of each user")
#fig = px.line(df_cluster_beer, x="style", y=1)
fig.show()
fig.write_html("centroid.html")

In [293]:
user_to_cat["first_50_cluster"] = our_clustering.labels_

In [294]:
user_to_cat

category,Bocks,Brown Ales,Dark Ales,Dark Lagers,Hybrid Beers,India Pale Ales,Pale Ales,Pale Lagers,Porters,Specialty Beer,Stouts,Strong Ales,Weird Cocktail,Wheat Beers,Wild/Sour Beers,first_50_cluster
user_name,,,,,,,,,,,,,,,,
-Rick,0.0,2.0,0.0,2.0,0.0,29.0,7.0,2.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,2
1000lbgrizzly,0.0,5.0,0.0,1.0,0.0,4.0,9.0,7.0,2.0,8.0,6.0,4.0,0.0,3.0,1.0,0
1424IpA,0.0,0.0,1.0,0.0,0.0,21.0,3.0,1.0,1.0,1.0,12.0,4.0,0.0,1.0,5.0,2
2Xmd,0.0,2.0,1.0,1.0,0.0,25.0,5.0,1.0,1.0,2.0,9.0,1.0,0.0,2.0,0.0,2
2ellas,0.0,2.0,1.0,0.0,0.0,31.0,9.0,0.0,0.0,0.0,2.0,4.0,0.0,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zenithberwyn,1.0,1.0,0.0,3.0,0.0,3.0,7.0,10.0,3.0,0.0,14.0,5.0,0.0,3.0,0.0,3
zhanson88,1.0,5.0,3.0,1.0,0.0,6.0,6.0,2.0,3.0,1.0,12.0,7.0,0.0,3.0,0.0,1
zookerman182,1.0,3.0,0.0,0.0,0.0,22.0,6.0,0.0,2.0,0.0,9.0,5.0,0.0,1.0,1.0,2


### cluster 50 last

In [295]:
df_to_cluster_last = df_reviews_final[df_reviews_final["rating_number"] > 450]
user_to_cat_last = df_to_cluster_last.groupby(["user_name", "category"])["beer_id"].count()
user_to_cat_last = user_to_cat_last.unstack(level=-1)
user_to_cat_last.fillna(0, inplace = True)

In [296]:
our_clustering_last = KMeans(4).fit(user_to_cat_last.values)
count_last = collections.Counter(our_clustering_last.labels_)
count_last

Counter({0: 202, 3: 393, 2: 258, 1: 117})

In [297]:
df_cluster_beer_last = pd.DataFrame(our_clustering_last.cluster_centers_.T)
#df_cluster_beer.rename(columns = user_to_cat.columns)
df_cluster_beer_last.index = user_to_cat_last.columns
df_cluster_beer_last["style"] = df_cluster_beer_last.index
df_cluster_beer_last = pd.melt(df_cluster_beer_last, id_vars=["style"], value_vars=[0,1,2,3])
df_cluster_beer_last.columns = ["style", "cluster", "Number of beer"]
df_cluster_beer_last

,style,cluster,Number of beer
0,Bocks,0,7.277228e-01
1,Brown Ales,0,2.014851e+00
2,Dark Ales,0,7.673267e-01
3,Dark Lagers,0,4.752475e-01
4,Hybrid Beers,0,2.178218e-01
5,India Pale Ales,0,9.455446e+00
6,Pale Ales,0,5.519802e+00
7,Pale Lagers,0,1.242574e+00
8,Porters,0,2.777228e+00
9,Specialty Beer,0,1.257426e+00


In [298]:
fig = px.line(df_cluster_beer_last, x="style", y="Number of beer", color="cluster", title="Centroids obtained with Kmeans (k=4) of the last 50 ratings of each user")
#fig = px.line(df_cluster_beer, x="style", y=1)
fig.show()
fig.write_html("centroid_last.html")

### compare first vs last

In [299]:
#user_to_cat["first_50_cluster"].to_pickle("user_to_cluster.pkl")
user_to_cat

category,Bocks,Brown Ales,Dark Ales,Dark Lagers,Hybrid Beers,India Pale Ales,Pale Ales,Pale Lagers,Porters,Specialty Beer,Stouts,Strong Ales,Weird Cocktail,Wheat Beers,Wild/Sour Beers,first_50_cluster
user_name,,,,,,,,,,,,,,,,
-Rick,0.0,2.0,0.0,2.0,0.0,29.0,7.0,2.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,2
1000lbgrizzly,0.0,5.0,0.0,1.0,0.0,4.0,9.0,7.0,2.0,8.0,6.0,4.0,0.0,3.0,1.0,0
1424IpA,0.0,0.0,1.0,0.0,0.0,21.0,3.0,1.0,1.0,1.0,12.0,4.0,0.0,1.0,5.0,2
2Xmd,0.0,2.0,1.0,1.0,0.0,25.0,5.0,1.0,1.0,2.0,9.0,1.0,0.0,2.0,0.0,2
2ellas,0.0,2.0,1.0,0.0,0.0,31.0,9.0,0.0,0.0,0.0,2.0,4.0,0.0,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zenithberwyn,1.0,1.0,0.0,3.0,0.0,3.0,7.0,10.0,3.0,0.0,14.0,5.0,0.0,3.0,0.0,3
zhanson88,1.0,5.0,3.0,1.0,0.0,6.0,6.0,2.0,3.0,1.0,12.0,7.0,0.0,3.0,0.0,1
zookerman182,1.0,3.0,0.0,0.0,0.0,22.0,6.0,0.0,2.0,0.0,9.0,5.0,0.0,1.0,1.0,2


In [300]:
mean_user = pd.DataFrame()

#IPA
#add first clust - IPA dora stout palelager
ipa_lovers = user_to_cat[user_to_cat["first_50_cluster"] == 0]
ipa_lovers.drop(columns = "first_50_cluster", inplace = True)
mean_user["first_50"] = ipa_lovers.mean()
#mean_user.drop( labels = "first_50_cluster", inplace = True)
#mean_user["category"] = mean_user.index
mean_user = mean_user.reset_index(level=0)

#add last clust
mean_user["last_50"] = user_to_cat_last[user_to_cat_last.index.isin(ipa_lovers.index)].mean().values

#melt
mean_user = pd.melt(mean_user, id_vars=["category"], value_vars=["first_50", "last_50"])
mean_user.columns = ["category", "first/last", "IPA"]

#DORA
all_lovers = user_to_cat[user_to_cat["first_50_cluster"] == 1]
all_lovers.drop(columns = "first_50_cluster", inplace = True)
mean_user["dora"] = np.concatenate([
    all_lovers.mean(),
    user_to_cat_last[user_to_cat_last.index.isin(all_lovers.index)].mean().values
])

#stout
stout_lovers = user_to_cat[user_to_cat["first_50_cluster"] == 2]
stout_lovers.drop(columns = "first_50_cluster", inplace = True)
mean_user["stout"] = np.concatenate([
    stout_lovers.mean(),
    user_to_cat_last[user_to_cat_last.index.isin(stout_lovers.index)].mean().values
])

#palelager
palelager_lovers = user_to_cat[user_to_cat["first_50_cluster"] == 3]
palelager_lovers.drop(columns = "first_50_cluster", inplace = True)
mean_user["palelager"] = np.concatenate([
    palelager_lovers.mean(),
    user_to_cat_last[user_to_cat_last.index.isin(palelager_lovers.index)].mean().values
])

#all
mean_user["all"] = mean_user.iloc[:,2:6].mean(axis=1)

#mean_user
#all_lovers
mean_user




,category,first/last,IPA,dora,stout,palelager,all
0,Bocks,first_50,1.659898,0.816327,0.598326,1.794326,1.217219
1,Brown Ales,first_50,3.281726,2.505102,1.732218,2.723404,2.560612
2,Dark Ales,first_50,1.654822,0.959184,0.615063,1.425532,1.163650
3,Dark Lagers,first_50,1.530457,0.632653,0.548117,3.347518,1.514686
4,Hybrid Beers,first_50,0.337563,0.280612,0.234310,0.382979,0.308866
5,India Pale Ales,first_50,8.192893,9.872449,22.832636,3.297872,11.048963
6,Pale Ales,first_50,9.408629,4.750000,7.468619,7.780142,7.351848
7,Pale Lagers,first_50,3.868020,1.295918,2.012552,13.624113,5.200151
8,Porters,first_50,2.296954,3.096939,1.351464,1.560284,2.076410
9,Specialty Beer,first_50,1.743655,1.341837,1.221757,1.418440,1.431422


In [301]:
# Create base figure
import plotly.express as px

fig = px.line(mean_user, x="category", y="IPA", color="first/last", title="mean in 50 first and last")

# fig.add_annotation(dict(text=corr_annotation(data['A'], data['B']),
#                         showarrow=False, 
#                         yref='paper', xref='paper',
#                         x=0.99, y=0.95))

buttons = []

buttons.append(dict(method='update',
                    label="Dora",
                    args=[{'x': [mean_user["category"]],
                           'y': [mean_user["dora"]],
                           'color' : [mean_user["first/last"]],
                           'visible': [True, True]},
                            {'xaxis': {'title': "hello"},
                            'yaxis': {'title': y},}]
                    )
                )

                # args=[{'x': [mean_user["category"]],
                #            'y': [mean_user["dora"]],
                #            'color' : mean_user["first/last"]},
                #             {'xaxis': {'title': x},
                #             'yaxis': {'title': y},}]
                #     )

# Update and show figure
fig.update_layout(updatemenus=[dict(buttons=buttons, direction='down', x=0.1, y=1.15)])

fig.show()

In [302]:
mean_user.iloc[0:15,0].values
mean_user

,category,first/last,IPA,dora,stout,palelager,all
0,Bocks,first_50,1.659898,0.816327,0.598326,1.794326,1.217219
1,Brown Ales,first_50,3.281726,2.505102,1.732218,2.723404,2.560612
2,Dark Ales,first_50,1.654822,0.959184,0.615063,1.425532,1.163650
3,Dark Lagers,first_50,1.530457,0.632653,0.548117,3.347518,1.514686
4,Hybrid Beers,first_50,0.337563,0.280612,0.234310,0.382979,0.308866
5,India Pale Ales,first_50,8.192893,9.872449,22.832636,3.297872,11.048963
6,Pale Ales,first_50,9.408629,4.750000,7.468619,7.780142,7.351848
7,Pale Lagers,first_50,3.868020,1.295918,2.012552,13.624113,5.200151
8,Porters,first_50,2.296954,3.096939,1.351464,1.560284,2.076410
9,Specialty Beer,first_50,1.743655,1.341837,1.221757,1.418440,1.431422


In [303]:
import plotly.graph_objects as go

# plotly setup
fig = go.Figure()

# Add one ore more traces
x = mean_user.iloc[0:15,0].values
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,6].values,name='first 50', visible=True, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,6].values,name='last 50', visible=True, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,2].values,name='first 50 beers', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,2].values,name='last 50 beers', visible=False, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,3].values,name='first 50', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,3].values,name='last 50', visible=False, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,4].values,name='first 50', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,4].values,name='last 50', visible=False, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,5].values,name='first 50', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,5].values,name='last 50', visible=False, showlegend=True))




fig.update_layout({'title' : '"Jean-Michel IPA" team : average number of beers rated from each category'})

# construct menus
updatemenus = [{
#                 'active':1,
                'buttons': [{'method': 'update',
                             'label': 'All together',
                             'args': [
                                      # 1. updates to the traces
                                      {'y': [mean_user.iloc[0:15,6], mean_user.iloc[15:30,6]],
                                       'name':['first 50 beers', 'last 50 beers'],
                                       'visible': True}, 
                                      
                                      # 2. updates to the layout
                                      {'title':'All together : average number of beers rated from each category'},
                                      
                                      # 3. which traces are affected 
                                      [0, 1],
                                      
                                      ],
                              },
                              {'method': 'update',
                             'label': 'Jean-Michel IPA',
                             'args': [
                                      # 1. updates to the traces
                                      {'y': [mean_user.iloc[0:15,2], mean_user.iloc[15:30,2]],
                                       'name':['first 50 beers', 'last 50 beers'],
                                       'visible': True}, 
                                      
                                      # 2. updates to the layout
                                      {'title':'"Jean-Michel IPA" team : average number of beers rated from each category'},
                                      
                                      # 3. which traces are affected 
                                      [0, 1],
                                      
                                      ],
                              },
                              {'method': 'update',
                             'label': 'Ada the explorer',
                             'args': [
                                      # 1. updates to the traces
                                      {'y': [mean_user.iloc[0:15,3], mean_user.iloc[15:30,3]],
                                       'name':['first 50 beers', 'last 50 beers'],
                                       'visible': True}, 
                                      
                                      # 2. updates to the layout
                                      {'title':'"Ada the explorer" team : average number of beers rated from each category'},
                                      
                                      # 3. which traces are affected 
                                      [0,1],
                                      
                                      ],
                              },
                              {'method': 'update',
                             'label': 'Marcel Stout',
                             'args': [
                                      # 1. updates to the traces
                                      {'y': [mean_user.iloc[0:15,4], mean_user.iloc[15:30,4]],
                                       'name':['first 50 beers', 'last 50 beers'],
                                       'visible': True}, 
                                      
                                      # 2. updates to the layout
                                      {'title':'"Marcel Stout" team : average number of beers rated from each category'
                                      ""},
                                      
                                      # 3. which traces are affected 
                                      [0, 1],
                                      
                                      ],
                              },
                              {'method': 'update',
                             'label': 'James Bland',
                             'args': [
                                      # 1. updates to the traces
                                      {'y': [mean_user.iloc[0:15,5], mean_user.iloc[15:30,5]],
                                       'name':['first 50 beers', 'last 50 beers'],
                                       'visible': True}, 
                                      
                                      # 2. updates to the layout
                                      {'title':'"James bland" team : average number of beers rated from each category'},
                                      
                                      # 3. which traces are affected 
                                      [0, 1],
                                      
                                      ],
                              }
                            ],
                            
                'type':'buttons',
                'type':'dropdown',
                'direction': 'down',
                'x' :0.19, 
                'y': 1.15,
                'showactive': True,}]

# update layout with buttons, and show the figure
fig.update_layout(updatemenus=updatemenus)
fig.show()
fig.write_html("all_first_vs_last.html")


In [304]:
# imports
import plotly.graph_objects as go
import numpy as np

# data
x = np.linspace(-np.pi, np.pi, 10)

y1 = np.sin(x)
y1b = y1-1

y2 = np.tan(x)
y2b = y2-1

# plotly setup
fig = go.Figure()

# Add one ore more traces
fig.add_traces(go.Scatter(x=x, y=y1,name='sin', visible=True, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=y1b,name='sin - 1', visible=True, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=y2,name='tan', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=y2b,name='tan - 1', visible=False, showlegend=True))
fig.update_layout({'title' : 'Sine'})

# construct menus
updatemenus = [{
#                 'active':1,
                'buttons': [{'method': 'update',
                             'label': 'Toggle Sine / Tangent',
                             'args': [
                                      # 1. updates to the traces
                                      {'y': [y1, y1b],
                                       'name':['sin', 'sin - 1'],
                                       'visible': True}, 
                                      
                                      # 2. updates to the layout
                                      {'title':'Sine'},
                                      
                                      # 3. which traces are affected 
                                      [0, 1],
                                      
                                      ],
                             'args2': [
                                       # 1. updates to the traces  
                                       {'y': [y2, y2b],
                                       'name':['tan', 'tan - 1'],
                                       'visible':True},
                                      
                                       # 2. updates to the layout
                                       {'title':'Tangent'},
                                       
                                       # 3. which traces are affected
                                       [0, 1]
                                      ]
                              },
                            ],
                'type':'buttons',
#                 'type':'dropdown',
                'direction': 'down',
                'showactive': True,}]

# update layout with buttons, and show the figure
fig.update_layout(updatemenus=updatemenus)
fig.show()


In [323]:
#LE VRAI

# plotly setup 
fig = go.Figure()

# Add one ore more traces
x = mean_user.iloc[0:15,0].values
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,6].values,name='first 50 beers', visible=True, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,6].values,name='last 50 beers', visible=True, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,2].values,name='first 50 beers', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,2].values,name='last 50 beers', visible=False, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,3].values,name='first 50 beers', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,3].values,name='last 50 beers', visible=False, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,4].values,name='first 50 beers', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,4].values,name='last 50 beers', visible=False, showlegend=True))

fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[0:15,5].values,name='first 50 beers', visible=False, showlegend=True))
fig.add_traces(go.Scatter(x=x, y=mean_user.iloc[15:30,5].values,name='last 50 beers', visible=False, showlegend=True))

list_0 = [True, True, False, False, False, False, False, False, False, False,]
list_1 = [False, False, True, True, False, False, False, False, False, False,]
list_2 = [False, False, False, False, True, True, False, False, False, False,]
list_3 = [False, False, False, False, False, False, True, True, False, False,]
list_4 = [False, False, False, False, False, False, False, False,True, True, ]

fig.update_layout(
    yaxis_range=[0,25],
    width=1000,
    height=600,
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=1,
            y=1.1,
            buttons=list([
                dict(label="All togheter",
                     method="update",
                     args=[{"visible": list_0},
                           {'title':'All together : average number of beers rated from each category'}]),
                dict(label="Jean-Michel IPA",
                     method="update",
                     args=[{"visible": list_1},
                           {'title' : '"Jean-Michel IPA" team : average number of beers rated from each category'}]),
                dict(label="Marcel Stout",
                     method="update",
                     args=[{"visible": list_2},
                           {'title' : '"Marcel Stout" team : average number of beers rated from each category'}]),
                dict(label="Ada the explorer",
                     method="update",
                     args=[{"visible": list_3},
                           {'title' : '"Ada the explorer" team : average number of beers rated from each category'}]),
                dict(label="James Bland",
                     method="update",
                     args=[{"visible": list_4},
                           {'title' : '"James Bland" team : average number of beers rated from each category'}]),
            ]),
        )
    ])

# Set title
fig.update_xaxes(title_text="Category")
fig.update_yaxes(title_text="Number of beers")

fig.update_layout({'title' : 'All together : average number of beers rated from each category'})


fig.show()
fig.write_html("ratio_cat_num_clusters.html")

In [ ]:
fig = go.Figure()

# Add Traces
for category, group in df_cat_year_0.groupby("category"):
    fig.add_trace(go.Line(
        name=category,
        x=group["rating_number"], 
        y=group["ratio"],
        visible=True
    ))
for category, group in df_cat_year_1.groupby("category"):
    fig.add_trace(go.Line(
        name=category,
        x=group["rating_number"], 
        y=group["ratio"],
        visible=False
    ))
for category, group in df_cat_year_2.groupby("category"):
    fig.add_trace(go.Line(
        name=category,
        x=group["rating_number"], 
        y=group["ratio"],
        visible=False
    ))
for category, group in df_cat_year_3.groupby("category"):
    fig.add_trace(go.Line(
        name=category,
        x=group["rating_number"], 
        y=group["ratio"],
        visible=False
    ))


fig.update_layout(
    yaxis_range=[0,0.5],
    width=1000,
    height=600,
    updatemenus=[
        dict(
            type="buttons",
            direction="right",

            active=0,
            x=1,
            y=1.1,
            buttons=list([
                dict(label="Jean-Michel IPA",
                     method="update",
                     args=[{"visible": list_0}]),
                dict(label="Marcel Stout",
                     method="update",
                     args=[{"visible": list_2}]),
                dict(label="Ada the explorer",
                     method="update",
                     args=[{"visible": list_1}]),
                dict(label="James Bland",
                     method="update",
                     args=[{"visible": list_3}]),
            ]),
        )
    ])

# Set title
fig.update_xaxes(title_text="rating number")
fig.update_yaxes(title_text="ratio")
fig.update_layout(

    title_text="Ratio of ratings per category and rating number",
    xaxis_domain=[0.05, 1.0]
)

fig.show()
fig.write_html("ratio_cat_num_clusters.html")

### was was tried on IPA (do not consider)

In [305]:
ipa_lovers = pd.DataFrame()
user_IPA = user_to_cat[user_to_cat["first_50_cluster"] == 1] #select cluster
user_IPA.drop(columns = "first_50_cluster", inplace = True)


ipa_lovers["first_50_IPA_mean"] = user_IPA.mean()
ipa_lovers


,first_50_IPA_mean
category,
Bocks,0.816327
Brown Ales,2.505102
Dark Ales,0.959184
Dark Lagers,0.632653
Hybrid Beers,0.280612
India Pale Ales,9.872449
Pale Ales,4.750000
Pale Lagers,1.295918
Porters,3.096939


In [306]:
ipa_lovers["last_50_IPA_mean"] = user_to_cat_last[user_to_cat_last.index.isin(user_IPA.index)].mean()


In [307]:
ipa_lovers

,first_50_IPA_mean,last_50_IPA_mean
category,,
Bocks,0.816327,0.535714
Brown Ales,2.505102,1.852041
Dark Ales,0.959184,0.581633
Dark Lagers,0.632653,0.540816
Hybrid Beers,0.280612,0.244898
India Pale Ales,9.872449,11.250000
Pale Ales,4.750000,6.867347
Pale Lagers,1.295918,1.607143
Porters,3.096939,2.556122


In [308]:
ipa_lovers["category"] = ipa_lovers.index
ipa_lovers = pd.melt(ipa_lovers, id_vars=["category"], value_vars=["first_50_IPA_mean", "last_50_IPA_mean"])
ipa_lovers.columns = ["category", "first/last", "Number_of_beer"]
ipa_lovers

,category,first/last,Number_of_beer
0,Bocks,first_50_IPA_mean,0.816327
1,Brown Ales,first_50_IPA_mean,2.505102
2,Dark Ales,first_50_IPA_mean,0.959184
3,Dark Lagers,first_50_IPA_mean,0.632653
4,Hybrid Beers,first_50_IPA_mean,0.280612
5,India Pale Ales,first_50_IPA_mean,9.872449
6,Pale Ales,first_50_IPA_mean,4.750000
7,Pale Lagers,first_50_IPA_mean,1.295918
8,Porters,first_50_IPA_mean,3.096939
9,Specialty Beer,first_50_IPA_mean,1.341837


In [309]:
fig = px.line(ipa_lovers, x="category", y="Number_of_beer", color="first/last", title="mean in 50 first and last")
#fig = px.line(df_cluster_beer, x="style", y=1)
fig.show()

### try all in one

In [310]:
from scipy import stats

def corr_annotation(x, y):
    pearsonr = stats.pearsonr(x, y)
    return 'r = {:.2f} (p = {:.3f})'.format(pearsonr[0], pearsonr[1])

# Prep random data
import pandas as pd
import numpy as np

np.random.seed(12)

data = pd.DataFrame(dict(
    A=np.random.randint(11, size=10),
    B=np.random.randint(11, size=10),
    C=np.random.randint(11, size=10),
    D=np.random.randint(11, size=10) 
))

# Create base figure
import plotly.express as px

fig = px.scatter(data, x='A', y='B')

# fig.add_annotation(dict(text=corr_annotation(data['A'], data['B']),
#                         showarrow=False, 
#                         yref='paper', xref='paper',
#                         x=0.99, y=0.95))

# Create buttons
import itertools

buttons = []

for x, y in itertools.combinations(data.columns, 2):
    buttons.append(dict(method='update',
                        label='{} x {}'.format(x, y),
                        args=[{'x': [data[x]],
                               'y': [data[y]]},
                              {'xaxis': {'title': x},
                               'yaxis': {'title': y},
                            #    'annotations': [dict(text=corr_annotation(data[x], data[y]),
                            #                         showarrow=False, 
                            #                         yref='paper', xref='paper',
                            #                         x=0.99, y=0.95)]}]
                        }]
                        )
                   )

# Update and show figure
fig.update_layout(updatemenus=[dict(buttons=buttons, direction='down', x=0.1, y=1.15)])

fig.show()
print(data)
buttons

   A   B  C  D
0  6   9  4  6
1  1   2  3  9
2  2   6  1  0
3  3  10  7  0
4  3   0  0  9
5  0   5  2  8
6  6   8  6  9
7  1   2  2  6
8  4   9  0  1
9  5   3  4  8


[{'method': 'update',
  'label': 'A x B',
  'args': [{'x': [0    6
     1    1
     2    2
     3    3
     4    3
     5    0
     6    6
     7    1
     8    4
     9    5
     Name: A, dtype: int32],
    'y': [0     9
     1     2
     2     6
     3    10
     4     0
     5     5
     6     8
     7     2
     8     9
     9     3
     Name: B, dtype: int32]},
   {'xaxis': {'title': 'A'}, 'yaxis': {'title': 'B'}}]},
 {'method': 'update',
  'label': 'A x C',
  'args': [{'x': [0    6
     1    1
     2    2
     3    3
     4    3
     5    0
     6    6
     7    1
     8    4
     9    5
     Name: A, dtype: int32],
    'y': [0    4
     1    3
     2    1
     3    7
     4    0
     5    2
     6    6
     7    2
     8    0
     9    4
     Name: C, dtype: int32]},
   {'xaxis': {'title': 'A'}, 'yaxis': {'title': 'C'}}]},
 {'method': 'update',
  'label': 'A x D',
  'args': [{'x': [0    6
     1    1
     2    2
     3    3
     4    3
     5    0
     6    6
     7    1
     8 